<a href="https://colab.research.google.com/github/phisan-chula/2021-LDP_Design/blob/main/LDP_Mini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Design of a Low Distortion Projection for a Mini Project***</br>  
LDP_Inspection : program to design  low distortion projections (LDPs) using conformal map projections for minimizing linear distortion between projected coordinates eg. UTM grid and the true distance at the surface of the engineering project.</br>  

Phisan Santitamonont,</br>  
Faculty of Engineering, Chulalongkorn University © 2022</br>
*Phisan.Chula@gmail.com*

*** 1) Installing RTSD/CMU Thai Geoid Model 2017 and Python modules ***

In [17]:
# download TGM-2017 from Khun Projuab Riabroy archive.
import urllib.request
from pathlib import Path
url = "https://www.priabroy.name/?sdm_process_download=1&download_id=8512"
TGM2017 = '/content/tgm2017-1.pgm'

if Path(TGM2017).exists():
    print(f'The file {TGM2017} exists.')
else:
    print(f'The file {TGM2017} does not exist, downloading...few minutes...')
    try:
        urllib.request.urlretrieve(url, TGM2017)
        print(f'File downloaded successfully and saved as {TGM2017}.')
    except Exception as e:
        print(f'Failed to download the file. Error: {e}')

The file /content/tgm2017-1.pgm does not exist, downloading...few minutes...
File downloaded successfully and saved as /content/tgm2017-1.pgm.


In [18]:
%%capture
! pip install pygeodesy
! pip install pyproj
! pip install requests
! pip install folium
! pip install toml

In [19]:
def dd2DMS( dd, PREC=7, POS=''  ):
    '''convert degree to DMS string'''
    return pgd.dms.toDMS( dd, prec=PREC,pos=POS )

def CalcLDP( row ):
    UNDUL = GEOID.height( row.lat,row.lng )
    RG   = ELLPS.rocGauss( row.lat )
    h     = UNDUL + row.MSL
    HSF = RG/(RG+h)
    PSF = pyproj.Proj( LDP).get_factors( row.lng, row.lat ).meridional_scale
    CSF = PSF*HSF
    CSF_ppm = (CSF-1)*1E6
    TR = pyproj.Transformer.from_crs( 'epsg:4326', LDP )
    LDP_E,LDP_N = TR.transform( row.lat, row.lng )
    return [UNDUL, h, HSF, PSF, CSF, CSF_ppm, LDP_E, LDP_N]

*** 2)  Define projection plane (PP) ***<br>
User input [lat,lng] ,horizontal , vertical  buffer --> topo(SRTM/MSL) height <br>
starting design central meridian degree, minute , false easting, false northing<br>

In [20]:
import toml
###########################################
#POS, HOR_BUF,  VER_BUF =[13.5411528, 99.8240430], 2_000, 20.0 # Rachburi,
#CM_DEG, CM_MIN, FALSE_E, FALSE_N = 99, 49, +2_500, -1_490_000
POS, HOR_BUF, VER_BUF = [14.519354, 101.017051], 1_000, 50  # CU-SBR
CM_DEG, CM_MIN, FALSE_E, FALSE_N = 101, 1, +2_500, -1_600_000
###########################################################

In [22]:
import requests
import pygeodesy as pgd
import numpy as np
from shapely.geometry import Point
import pandas as pd
import pyproj

COL_LDP = ['UNDUL', 'h','HSF','PSF','CSF', 'CSF_ppm', 'LDP_E', 'LDP_N']
FLT_MAP =  { 'MSL': '{0:.1f}', 'UNDUL': '{0:.1f}', 'h': '{0:.1f}', 'CSF_ppm': '{0:.1f}',
                     'LDP_E': '{0:,.3f}',  'LDP_N': '{0:,.3f}' }
TM = r'+proj=tmerc +lat_0=0.0 +lon_0={} +k_0={}  +x_0={}  +y_0={}  +a={} +b={} +units=m +no_defs'
###########################################################
res = requests.get( r'https://api.opentopodata.org/v1/srtm30m?locations={},{}'.format( *POS ) )
MSL = res.json()['results'][0]['elevation']
ELLPS  = pgd.datums.Ellipsoids.WGS84
GEOID = pgd.geoids.GeoidKarney( TGM2017 )

NameError: name 'TGM2017' is not defined

*** 3) Calculation Low Distortion Projection via Transverse Mercatior (TM)***<br>

In [ ]:
res = requests.get( r'https://api.opentopodata.org/v1/srtm30m?locations={},{}'.format( *POS ) )
MSL = res.json()['results'][0]['elevation']
####################################################
UNDUL = GEOID.height( *POS )
HAE = UNDUL + MSL                 # h = N + H
RG = ELLPS.rocGauss( POS[0] )  # RG = sqrt(MN)
k0 = np.round(1 + HAE/RG, 6)    #  M.Dennis 2016 : Ground Truth ... (...5 to 6 digits)
print( f'Projection Plane    : lat = {dd2DMS(POS[0]):}  lng = {dd2DMS(POS[1]):}  ==> k0 = {k0:.6f} ')
print( f'Topography          :  MSL = {MSL:.1f} m. HAE={HAE:.1f} m. ,  N={UNDUL:.1f} m.  ' )
print( f'''Designed LDP-TM :  {CM_DEG:.0f} deg {CM_MIN:.0f} min '''\
         f'''    FALSE_Easting={FALSE_E:+,.0f} m.   FALSE_Northing={FALSE_N:+,.0f} m ''' )
LDP = pyproj.CRS( TM.format( CM_DEG+CM_MIN/60., k0, FALSE_E, FALSE_N, ELLPS.a, ELLPS.b ) )
print(  f'{LDP}' )

Projection Plane    : lat = 14°31′09.6744″  lng = 101°01′01.3836″  ==> k0 = 1.000000 
Topography          :  MSL = 29.0 m. HAE=-0.7 m. ,  N=-29.7 m.  
Designed LDP-TM :  101 deg 1 min     FALSE_Easting=+2,500 m.   FALSE_Northing=-1,600,000 m 
+proj=tmerc +lat_0=0.0 +lon_0=101.01666666666667 +k_0=1.0  +x_0=2500  +y_0=-1600000  +a=6378137.0 +b=6356752.314245179 +units=m +no_defs +type=crs


*** 4) Test representative points with the designated LDP ***<br>
create 3 rectangular planes with 5 representative points and test them

In [ ]:
from IPython.display import display
EWNS =Point( POS[1],POS[0] ).buffer( HOR_BUF/111_000 , cap_style = 3  ).exterior.coords.xy
PP_EWNS = np.vstack( (np.array( [POS[1],POS[0]] ) , np.array(EWNS).T) )[:-1]
dfPP = pd.DataFrame( {'Point':['P0','P1','P2','P3','P4'], 'lng':PP_EWNS[:,0], 'lat':PP_EWNS[:,1]  } )
dfPP = pd.concat( 3*[dfPP] ,  ignore_index=True) # create 3 planes ...
dfPP['MSL'] = 5*[MSL+VER_BUF]+  5*[MSL] +  5*[MSL-VER_BUF]
dfPP[COL_LDP] = dfPP.apply( CalcLDP, axis=1, result_type='expand')
display( dfPP.style.format(FLT_MAP) )

,Point,lng,lat,MSL,UNDUL,h,HSF,PSF,CSF,CSF_ppm,LDP_E,LDP_N
0,P0,101.017051,14.519354,79.0,-29.7,49.3,0.999992,1.000000,0.999992,-7.7,"2,541.426","5,807.846"
1,P1,101.026060,14.528363,79.0,-29.7,49.3,0.999992,1.000000,0.999992,-7.7,"3,512.439","6,804.661"
2,P2,101.026060,14.510345,79.0,-29.7,49.3,0.999992,1.000000,0.999992,-7.7,"3,512.521","4,811.073"
3,P3,101.008042,14.510345,79.0,-29.8,49.2,0.999992,1.000000,0.999992,-7.7,"1,570.335","4,811.070"
4,P4,101.008042,14.528363,79.0,-29.8,49.2,0.999992,1.000000,0.999992,-7.7,"1,570.410","6,804.657"
5,P0,101.017051,14.519354,29.0,-29.7,-0.7,1.000000,1.000000,1.000000,0.1,"2,541.426","5,807.846"
6,P1,101.026060,14.528363,29.0,-29.7,-0.7,1.000000,1.000000,1.000000,0.1,"3,512.439","6,804.661"
7,P2,101.026060,14.510345,29.0,-29.7,-0.7,1.000000,1.000000,1.000000,0.1,"3,512.521","4,811.073"
8,P3,101.008042,14.510345,29.0,-29.8,-0.8,1.000000,1.000000,1.000000,0.1,"1,570.335","4,811.070"
9,P4,101.008042,14.528363,29.0,-29.8,-0.8,1.000000,1.000000,1.000000,0.1,"1,570.410","6,804.657"


*** 5) User may input project's control points, RTKs, setting-out coordinates here... ***<br>
****

In [ ]:
from io import StringIO
CSV = '''Point,lat,lng,MSL
RTK1,13.542153,99.824043,20.0
RTK2,13.540153,99.821043,-30.0
RTK3,13.541153,99.822043,2.0
'''
CALC_RTK = False
if CALC_RTK:
    dfRTK = pd.read_csv( StringIO(CSV) )
    dfRTK[COL_LDP] = dfRTK.apply( CalcLDP, axis=1, result_type='expand')
    display( dfRTK.style.format(FLT_MAP) )

*** 6) Plot all test points and project RTK/CP points with their CSF ... ***<br>

In [ ]:
import folium
from folium.features import DivIcon

icon=folium.Icon(color='red', icon='plus' )
map = folium.Map(location =[POS[0],POS[1]],  zoom_start = 14 )
for grp,row in dfPP.groupby( [ 'Point' ] ):
    csf = list( row.CSF_ppm.round(1) ); pnt = '{:}'.format( row.iloc[0]['Point'] )
    location = (row.iloc[0].lat , row.iloc[0].lng)
    folium.CircleMarker(location=location, radius=15, color='red', fill_color ='red', fill_opacity=0.5,
                                tooltip=f'<b>CSF:{csf:}</b>').add_to(map)
    folium.map.Marker(location, icon=DivIcon( icon_size=(30,30), icon_anchor=(5,14),
                          html=f'<div style="font-size: 14pt">{pnt:}</div>' ) ).add_to(map)
if CALC_RTK:
    for i,row in dfRTK.iterrows():
        location = (row.lat,row.lng) ; pnt = '{}:[{}m]'.format( row['Point'], row['MSL'] )
        folium.CircleMarker(location=location, tooltip=f'<b>CSF:{row.CSF_ppm:}</b>',
                            radius=15, color='green', fill_color ='green', fill_opacity=0.5 ).add_to(map)
        folium.map.Marker(location, icon=DivIcon( icon_size=(30,30), icon_anchor=(5,14),
                       html=f'<div class="center" style="font-size: 16p"><p>{pnt:}</p></div>'  ) ).add_to(map)
map